In [0]:
#go to kaggle account setting and generate authentication token
#and download the kaggle.json file and then upload it here

# Run this and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Next, install the Kaggle API client.
!pip install kaggle

# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 /content/.kaggle/kaggle.json

# List available datasets.
# !kaggle datasets list

# Copy the stackoverflow data set locally.
!kaggle competitions download -c tgs-salt-identification-challenge

Saving kaggle.json to kaggle.json
-rw-r--r-- 1 root root 71 Aug 22 04:13 kaggle.json
    100% |████████████████████████████████| 51kB 2.3MB/s 
    100% |████████████████████████████████| 51kB 5.5MB/s 
  Running setup.py bdist_wheel for kaggle ... - done
  Stored in directory: /root/.cache/pip/wheels/56/65/d9/88f616157b8a1848ce77c35a398b09d0566ba1fd9cc34eaa03
Successfully built kaggle
chmod: cannot access '/content/.kaggle/kaggle.json': No such file or directory
  0%|                                                | 0.00/322k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 322k/322k [00:00<00:00, 49.4MB/s]
  0%|                                                | 0.00/264k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 264k/264k [00:00<00:00, 36.4MB/s]
  0%|                                                | 0.00/922k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 922k/922k [00:00<00:00, 47.3MB/s]
 93%|██████████████████████████████████████▏  

In [0]:
!unzip train.zip -d train
# !unzip test.zip -d test


Archive:  train.zip
   creating: train/images/
   creating: train/masks/
  inflating: train/images/8d08955cdf.png  
  inflating: train/images/e045664b46.png  
  inflating: train/images/4989c6509e.png  
  inflating: train/images/fff4eb4941.png  
  inflating: train/images/3a310860a3.png  
  inflating: train/images/7a696bb878.png  
  inflating: train/images/8d3ee9356b.png  
  inflating: train/images/213e0a9f7b.png  
  inflating: train/images/78f17697f2.png  
  inflating: train/images/bd20d18369.png  
  inflating: train/images/de08c5a3b7.png  
  inflating: train/images/f06ef6c8b7.png  
  inflating: train/images/b2d02516e1.png  
  inflating: train/images/5edb37f5a8.png  
  inflating: train/images/b1f0e104f5.png  
  inflating: train/images/40e8e14e60.png  
  inflating: train/images/6e1444d88d.png  
  inflating: train/images/f342603c76.png  
  inflating: train/images/7ba58ee1be.png  
  inflating: train/images/14a89a1840.png  
  inflating: train/images/4319885cf1.png  
  inflating: train/image

In [0]:
!pip install imageio
import imageio
import os
import numpy as np
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split
from math import pi

datalab     kaggle.json  sample_submission.csv	test.zip   train.zip
depths.csv  sample_data  test			train.csv


In [0]:
extension = ".png"
train_images = []
test_images = []
train_masks = []
train_paths = os.listdir("train/images")
test_paths = os.listdir("test/images")
mask_paths = os.listdir("train/images")
for i in train_paths:
    if i.endswith(extension):
        train_images.append(list(imageio.imread("train/images/"+i)))

for i in test_paths:
    if i.endswith(extension):
        test_images.append(list(imageio.imread("test/images/"+i)))

for i in mask_paths:
    if i.endswith(extension):
        train_masks.append(list(imageio.imread("train/masks/"+i)))

        
train_images = np.array(train_images)
new_images = np.array(test_images)
train_masks = np.array(train_masks)

In [0]:
print train_images.shape
print new_images.shape
print train_masks.shape

(4000, 101, 101, 3)
(0,)
(4000, 101, 101)


In [0]:
for i in range(len(test_paths)):
    test_paths[i] = test_paths[i][:-4]

In [0]:
#train_test_split
#save validation, test and new images set in their own seperate files
train_images,test_images,train_masks,test_masks = train_test_split(train_images,train_masks,train_size = 0.9, random_state = 0)
validation_images,test_images,validation_masks,test_masks = train_test_split(test_images,test_masks,train_size = 0.5, random_state = 0)
print train_images.shape
print train_masks.shape
print test_images.shape
print test_masks.shape
print validation_images.shape
print validation_masks.shape

with open("validation.pkl","w") as f:
    pickle.dump([validation_images,validation_masks],f)    

with open("train.pkl","w") as f:
    pickle.dump([train_images,train_masks],f)

with open("new.pkl","w") as f:
    pickle.dump([new_images,test_paths],f)
    
    

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#overwriting file in a particular folder in drive
def upload_to_google_drive(filename,folder_id):
    uploaded = False
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
    try:
        for file1 in file_list:
            if (file1['title'] == filename) and (not uploaded):
                file1.Delete()
                f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id, }]})
                f.SetContentFile(filename)
                f.Upload()
                uploaded = True
        if not uploaded:
            f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id, }]})
            f.SetContentFile(filename)
            f.Upload()
            uploaded = True
    except:
        print "Failed"
        
    return f.get("id")

folder = "1QP_49FZ0rfrUWHMcuB67H0ThF3XR7oZh"
upload_to_google_drive("validation.pkl",folder)
upload_to_google_drive("train.pkl",folder)
upload_to_google_drive("new.pkl",folder)